In [ ]:
# this code can only be run if xspec and models created specifically for xspec are present.  
import time
import xspec 
import numpy as np
import emcee
from multiprocessing import Pool
import matplotlib.pyplot as plt
import pickle

# it needs model which is hard coded in the script below, so it can not be run.
# will give an error something like — model relxillCp not available (no way out of this)
# the data directory is about 400 Mb, so I am not attaching the data directory.
xspec.Xset.restore("5660010101/xti/event_cl/fitted_files/analysis0_rexillCp_manual_chisqgti1.xcm")
initial_model = xspec.AllModels(1)

par_val = [5,6,14,15,20,24,25,26] # thawed parameter number in the model. Example: 6th parameter is gamma of nthcomp.

def log_Likelihood(params):
    model = xspec.AllModels(1) # load the current model
    # update each model parameter
    for i in range(len(par_val)):
        model(par_val[i]).values = params[i]
    # calculate updated statistic and likelihood for those model parameters
    cstat = xspec.Fit.statistic
    log_likelihood = -0.5*cstat
    return log_likelihood

def log_Posterior(params):
    # Defined for a uniform prior between parameter range.
    # if any parameter is outside range, the prior value = 0, log(prior) = -inf, thus, log(Posterior) = -inf
    for i in range(len(par_val)):
        pval, pdelta, pmin, pbottom, ptop, pmax = initial_model(par_val[i]).values # the range is same throughout
        # define a box-prior with the param range (in logarithm)
        if params[i] < pmin or params[i] > pmax: 
            return -np.inf # log(0)
    # if all parameters are within the range, the prior value = 1, log(prior) = 0, log(posterior) = log(likelihood) + 0
    log_posterior = log_Likelihood(params)
    return log_posterior

# Run MCMC
ndim = len(par_val)  # number of parameters in the model
nwalkers = 256  # number of MCMC walkers
burn = 500  # "burn-in" period to let chains stabilize
nsteps = 10000  # number of MCMC steps to take **for each walker**

# initialize theta 

best_fit = np.zeros(len(par_val))
delta_values = np.zeros(len(par_val))
for i in range(len(par_val)):
    best_fit[i] = initial_model(par_val[i]).values[0]
    delta_values[i] = initial_model(par_val[i]).values[1]

starting_guesses = np.random.normal(loc=best_fit, scale=5*delta_values, size=(nwalkers, ndim))

# the function call where all the work happens: 
start = time.time()
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_Posterior,pool=pool)
    start = time.time()    
    sampler.run_mcmc(starting_guesses, nsteps)
    end = time.time()
    print("Time to run 100 iterations =", end-start,"s")
    from multiprocessing import cpu_count
    ncpu = cpu_count()
    print("{0} CPUs".format(ncpu))

# save the sampler
print("Saving data")
filename = f"sampler_w{nwalkers}_s{nsteps}.pkl"
with open(filename, "wb") as f:
    pickle.dump(sampler, f)


# sampler.chain is of shape (nwalkers, nsteps, ndim)
# throw-out the burn-in points and reshape:
emcee_trace  = sampler.chain[:, burn:, :].reshape(-1, ndim)
np.save("emcee_trace_w{nwalkers}_s{nsteps}.npy", emcee_trace)

print("done")

print(sampler.chain.shape) #original chain structure
print(emcee_trace.shape) #burned and flattened chain

import corner
import matplotlib.pyplot as plt
import xspec

# emcee_trace shape: (n_samples, len(par_val))
nparams = len(par_val)
assert emcee_trace.shape[1] == nparams, "Mismatch in number of parameters"

param_labels = [initial_model(par).name for par in par_val]

# Corner plot
figure = corner.corner(
    emcee_trace,
    labels=param_labels,
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    title_fmt=".4f",
    title_kwargs={"fontsize": 12}
)

plt.savefig('emcee_p_w{nwalkers}_s{nsteps}.png',dpi=300)
print("Congrats, job done.")